In [1]:
%pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [2]:
train = '1HqceFWkRXwEqgPcuAtACFVvHhKO_TGG6'
val = '1XEsLOUFJTnCGcitk4IUS23H6IlzCzwqZ'
test = '1Lu5ItQvj2iGRqMXJ-bovQkys0eilR9HZ'

train_embeddings_per_word = '1-0Bb3pLy_EhEggA-VsOh8TIS92VbmmxV'
val_embeddings_per_word = '1-0WpebJIz9q2PZ_Baf51ZUsiBx9pWVJP'
test_embeddings_per_word = '1pi6DGhZ7AoGzC3cqP-ET5Qvkc3HD33bo'

train_embeddings_per_sentence = '1-260zeDhoDdxQ3McfR7Hr4c6mfaLFBpa'
val_embeddings_per_sentence = '1-2EN_l5NcdgJZ740Szt4g6RcIZ9GX_D2'
test_embeddings_per_sentence = '1gNKahNHussBAV-6mFyC66AxPTA0YhQAG'

In [3]:
!gdown {train}
!gdown {val}
!gdown {test}

!gdown {train_embeddings_per_word}
!gdown {val_embeddings_per_word}
!gdown {test_embeddings_per_word}

Downloading...
From: https://drive.google.com/uc?id=1HqceFWkRXwEqgPcuAtACFVvHhKO_TGG6
To: /kaggle/working/train_final.pkl
100%|███████████████████████████████████████| 3.47M/3.47M [00:00<00:00, 191MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XEsLOUFJTnCGcitk4IUS23H6IlzCzwqZ
To: /kaggle/working/val_final.pkl
100%|█████████████████████████████████████████| 743k/743k [00:00<00:00, 111MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Lu5ItQvj2iGRqMXJ-bovQkys0eilR9HZ
To: /kaggle/working/test_final.pkl
100%|█████████████████████████████████████████| 737k/737k [00:00<00:00, 125MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1-0Bb3pLy_EhEggA-VsOh8TIS92VbmmxV
From (redirected): https://drive.google.com/uc?id=1-0Bb3pLy_EhEggA-VsOh8TIS92VbmmxV&confirm=t&uuid=816eabb8-6f49-4373-b569-a1dedc3a7ff5
To: /kaggle/working/train_embeddings_per_word.pkl
100%|███████████████████████████████████████| 1.08G/1.08G [00:03<00:00, 292MB/s]
Downloading...
From (uriginal)

In [4]:
import pandas as pd
import nltk
import string
import re
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.isri import ISRIStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from sklearn.svm import SVC
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import torch
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from collections import OrderedDict
import matplotlib.pyplot as plt
from datasets import load_dataset
from sklearn.preprocessing import OneHotEncoder

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [5]:
label_to_class = {
    0: 'none',
    1: 'anger',
    2: 'joy',
    3: 'sadness',
    4: 'love',
    5: 'sympathy',
    6: 'surprise',
    7: 'fear'
}
classes = ['none', 'anger', 'joy', 'sadness', 'love', 'sympathy', 'surprise', 'fear']

In [6]:
with open('/kaggle/working/train_final.pkl', 'rb') as f:
    train = pickle.load(f)
    
with open('/kaggle/working/val_final.pkl', 'rb') as f:
    val = pickle.load(f)

with open('/kaggle/working/test_final.pkl', 'rb') as f:
    test = pickle.load(f)

with open('/kaggle/working/train_embeddings_per_word.pkl', 'rb') as f:
    train_embeddings = pickle.load(f)
    
with open('/kaggle/working/val_embeddings_per_word.pkl', 'rb') as f:
    val_embeddings = pickle.load(f)
    
with open('/kaggle/working/test_embeddings_per_word.pkl', 'rb') as f:
    test_embeddings = pickle.load(f)

In [7]:
train_embeddings.shape

(7045, 50, 768)

In [9]:
# X_train = train_embeddings.reshape(train_embeddings.shape[0], 1, train_embeddings.shape[1]) # reshape input to allow for GRU
# X_val = val_embeddings.reshape(val_embeddings.shape[0], 1, val_embeddings.shape[1]) # reshape input to allow for GRU
# X_test = test_embeddings.reshape(test_embeddings.shape[0], 1, test_embeddings.shape[1]) # reshape input to allow for GRU

In [10]:
X_train = train_embeddings
X_val = val_embeddings
X_test = test_embeddings

In [11]:
encoder = OneHotEncoder()

y_train = encoder.fit_transform(train['label'].values.reshape(-1,1)).toarray()
y_val = encoder.fit_transform(val['label'].values.reshape(-1,1)).toarray()
y_test = encoder.transform(test['label'].values.reshape(-1,1)).toarray()

In [12]:
BATCH_SIZE = 128
EPOCHS = 50
EMBED_SIZE = 300
LEARNING_RATE =  0.001

early_stopping_monitor = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True)

In [13]:
def create_gru_model(model_type):
    model = Sequential()
#     model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=max_length, trainable=True))
    if model_type==1:
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True)))
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, return_sequences=True)))
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=False)))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(8, activation='softmax'))
    elif model_type==2:
        model.add(tf.keras.layers.GRU(512, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
        model.add(tf.keras.layers.GRU(256, return_sequences=True))
        model.add(tf.keras.layers.GRU(128, return_sequences=False))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(8, activation='softmax'))
    elif model_type==3:
        model.add(tf.keras.layers.GRU(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(8, activation='softmax'))
    elif model_type==4:
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False)))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(8, activation='softmax'))
    elif model_type==5:
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False)))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(8, activation='softmax'))
    
    opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    # model.summary()

    history = model.fit(X_train, np.asarray(y_train), validation_data=(X_val, np.asarray(y_val)), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping_monitor])
    return model, history

In [15]:
for i in range(1,6):
    model, _ = create_gru_model(i)
    predictions = model.predict(X_test)
    print('--------------------------------------------------------')
    print()
    print('CLassification report for model {}: '.format(i))
    print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1), target_names=classes))
    print()
    print('--------------------------------------------------------')

Epoch 1/50
56/56 [==============================] - 23s 168ms/step - loss: 2.1115 - accuracy: 0.1750 - val_loss: 1.9945 - val_accuracy: 0.2636
Epoch 2/50
56/56 [==============================] - 6s 116ms/step - loss: 1.9327 - accuracy: 0.2531 - val_loss: 1.7379 - val_accuracy: 0.3748
Epoch 3/50
56/56 [==============================] - 6s 116ms/step - loss: 1.7978 - accuracy: 0.3182 - val_loss: 1.6867 - val_accuracy: 0.3907
Epoch 4/50
56/56 [==============================] - 7s 117ms/step - loss: 1.7468 - accuracy: 0.3459 - val_loss: 1.5653 - val_accuracy: 0.4477
Epoch 5/50
56/56 [==============================] - 6s 116ms/step - loss: 1.6478 - accuracy: 0.3882 - val_loss: 1.5835 - val_accuracy: 0.4225
Epoch 6/50
56/56 [==============================] - 6s 115ms/step - loss: 1.6000 - accuracy: 0.4135 - val_loss: 1.5255 - val_accuracy: 0.4371
Epoch 7/50
56/56 [==============================] - 7s 116ms/step - loss: 1.5683 - accuracy: 0.4244 - val_loss: 1.5295 - val_accuracy: 0.4450
Epoch